# Ejercicio 2

Para esta seccion, seleccionamos de nuevo el corpus **cess_es** y lo preprocesamos de la misma manera que en el ejercicio 1

In [2]:
import re
import os
import pandas as pd
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import math   

from preprocesador import *
from subword import *
from frecuencias import *

os.environ["NLTK_DATA"]="datos_nltk/"
import  nltk
from nltk.corpus import stopwords

## Paso 1
Preprocesamos el corpus

In [3]:
## Seleccionamos el corpus
from nltk.corpus import cess_esp

nltk.download("cess_esp")

## Obtenemos las oraciones del corpus para el entrenamiento de nuestra red neuronal
corpus_sents = cess_esp.sents()[0:1000]

print("Numero de oraciones en el corpus:", len(corpus_sents))

[nltk_data] Downloading package cess_esp to /home/joel/nltk_data...
[nltk_data]   Package cess_esp is already up-to-date!


Numero de oraciones en el corpus: 1000


In [4]:
# Procedemos a normalizar el texto:
nltk.download('stopwords')
stopwords_list = stopwords.words('spanish')

# Normalizamos los oraciones
sents_norm = [pre_procesar(oracion, stopwords_list) for oracion in corpus_sents]
# Obtenemos todas las palabras del texto
tokens_normalizados = [x for xs in sents_norm for x in xs]

print("Numero de tokens, despues del preprocesamiento:", len(tokens_normalizados))

[nltk_data] Downloading package stopwords to /home/joel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Numero de tokens, despues del preprocesamiento: 17801


In [5]:
# Entrenamos nuestro algoritmo de BPE con las palabras del corpus, 
# obtenemos las reglas y las frecuencias de cada token
freqs_bpe, reglas_bpe = entrenar_byte_pair_encoding(tokens_normalizados, 500)

In [34]:
# Aplicamos nuestro modelo bpe a cada oracion para obtener las oraciones tokenizadas
sents_tokenizadas = [aplicar_bpe(sent, reglas_bpe) for sent in sents_norm]

# Agregamos la etiqueta <BOS> a cada oracion
sents_tokenizadas = [["<BOS>"] + sent for sent in sents_tokenizadas]


In [35]:
# Construimos la tabla para las frecuencias de las palabras
tabla_frecuencias_bpe = crear_tabla_frecuencias(freqs_bpe)
print(tabla_frecuencias_bpe)

         palabra  frecuencia
0             de         523
1              a         432
2              o         429
3             to         334
4             do         289
...          ...         ...
4226   lazarocar           1
4227  realidades           1
4228       queri           1
4229       fatal           1
4230       pleta           1

[4231 rows x 2 columns]


In [36]:
#Convertimos la tabla en un diccionario para usarlo en el modelo
palabras = tabla_frecuencias_bpe['palabra']
frecuencias = tabla_frecuencias_bpe['frecuencia']
dic_palabras = {}  
for i in range(len(frecuencias)):
    dic_palabras[palabras[i]] = frecuencias[i]
dic_palabras["<BOS>"] = len(sents_norm)
dic_palabras["<EOS>"] = len(sents_norm)

In [37]:
ETIQUETA_UNK = "<UNK>"
def indices_palabras(palabras_frecuencias: dict):
    """Calcula los indices de las palabras dadas sus frecuencias

    Parameters
    ----------
    palabras_frecuencias : dict
        Diccionario donde las llaves son las palabras y los valores sus frecuencias

    Returns
    -------
    dict,dict
        Diccionarios uno a uno de palabras a indices y viceversa
    """
    result = {}
    for ind, palabra in enumerate(palabras_frecuencias.keys()):
        # Ocurrio un Happax legomena o singleton
        if palabras_frecuencias[palabra] == 1:
            # Indice temporal para UNK
            result[ETIQUETA_UNK] = len(palabras_frecuencias)
        else:
            result[palabra] = ind

    return {word: idx for idx, word in enumerate(result.keys())}, {idx: word for idx, word in enumerate(result.keys())}

In [38]:
#Se obtienen los índices de cada palabra
palabras_a_indices, indices_a_palabras = indices_palabras(dic_palabras)
print(palabras_a_indices)
print(indices_a_palabras)
len(palabras_a_indices)

{'de': 0, 'a': 1, 'o': 2, 'to': 3, 'do': 4, 'co': 5, 'con': 6, 're': 7, 'ta': 8, 'c': 9, 'i': 10, 'ca': 11, 'ti': 12, 'te': 13, 'e': 14, 'en': 15, 'al': 16, 'es': 17, 'cia': 18, 'l': 19, 'fi': 20, 'da': 21, 'si': 22, 'di': 23, 'ex': 24, 'd': 25, 'mo': 26, 'z': 27, 'pe': 28, 's': 29, 'li': 30, 'h': 31, '2': 32, 'os': 33, 'hoy': 34, 'f': 35, '1': 36, 'se': 37, 'on': 38, 't': 39, 'p': 40, 'g': 41, 'le': 42, 'pro': 43, 'ci': 44, 'dos': 45, 'k': 46, 'gi': 47, 'ra': 48, 'bi': 49, 'in': 50, 'tar': 51, 'mi': 52, 'que': 53, 'r': 54, 'pi': 55, 'ar': 56, 'ri': 57, 'gobierno': 58, 'vi': 59, 'su': 60, '4': 61, 'y': 62, 'presidente': 63, 'ce': 64, '6': 65, 'car': 66, 'tas': 67, 'an': 68, 'b': 69, 'des': 70, 'tos': 71, 'go': 72, 'por': 73, '3': 74, 'hi': 75, 'u': 76, 'as': 77, 'com': 78, 'tes': 79, 'er': 80, 'ro': 81, 'dad': 82, 'ter': 83, 'la': 84, 'fe': 85, 'ento': 86, 'vo': 87, '7': 88, 've': 89, '0': 90, 'no': 91, 'ni': 92, 'cion': 93, 'zo': 94, 'cu': 95, 'so': 96, 'me': 97, 'ver': 98, 'per': 99,

2593

In [39]:
def get_id(palabra_a_id: dict, palabra: str) -> int:
    """Obtiene el id de una palabra dada

    Si no se encuentra la palabra se regresa el id
    del token UNK

    Parameters
    ----------
    palabra_a_id : dict
        Diccionario de palabras a indices
    palabra : str
        Palabra a buscar

    Returns
    -------
    int
        Indice de la palabra
    """
    id_unknow = palabra_a_id[ETIQUETA_UNK]
    return palabra_a_id.get(palabra, id_unknow)

In [44]:
def lista_indices(corpus:list, palabras_a_indices:dict) -> list:
    """Genera una lista de indices que representan las palabras de un corpus 
    Parameters
    ----------
    corpus : list
        corpus a procesar
    indices_a_palabras : dict
        Diccionario de palabras a indices
    Returns
    -------
    list
        Lista de indices que representan las palabras del corpus
    """
    indices = []
    for sent in corpus:
        renglon = []
        for token in sent:
            renglon.append(get_id(palabras_a_indices, token))
        indices.append(renglon)
    return indices

In [57]:
def lista_siguientes(entradas : list) -> list:
    """
    Genera una lista de palabras siguientes a partir de una lista de palabras.
    Parameters
    ----------
    entradas : list
        lista de listas de palabras, donde cada lista tiene 
        como primer elemento la etiqueta <BOS>.
    Returns
    -------
    list
        Lista de listas donde cada palabra es la palabra que le sigue 
        en la entrada, o la etiqueta <EOS> si es la última de la lista.
    """
    salidas = []
    for sent in entradas:
        renglon = []
        for i in range(len(sent)-1):
            renglon.append(sent[i+1])    
        renglon.append("<EOS>")                
        salidas.append(renglon)
    return(salidas)

In [63]:
#Obtenemos los índices de cada oración
indices_sents = lista_indices(sents_tokenizadas, palabras_a_indices)
print(indices_sents[0])
print(sents_tokenizadas[0])

[2591, 270, 17, 1437, 919, 175, 44, 13, 0, 191, 64, 14, 25, 35, 519, 34, 159, 241, 78, 970, 117, 36, 147, 293, 742, 336, 919, 175, 44, 2590, 2590, 16, 801, 48, 14, 1, 1, 151, 737, 2338, 52, 39, 2339, 75, 249, 2340, 1475, 102, 145, 1216, 917, 61, 123, 117, 97, 129, 1650]
['<BOS>', 'grupo', 'es', 'tatal', 'elec', 'tri', 'ci', 'te', 'de', 'fran', 'ce', 'e', 'd', 'f', 'anuncio', 'hoy', 'jue', 'ves', 'com', 'pra', '5', '1', 'porciento', 'empresa', 'mexi', 'cana', 'elec', 'tri', 'ci', 'dadagui', 'lade', 'al', 'tami', 'ra', 'e', 'a', 'a', 'cre', 'ada', 'japones', 'mi', 't', 'subis', 'hi', 'cor', 'poration', 'poneren', 'mar', 'cha', 'central', 'gas', '4', '9', '5', 'me', 'ga', 'vatios']


In [62]:
#Obtenemos las palabras siguientes y sus índices
siguientes_tokenizadas = lista_siguientes(sents_tokenizadas)
indices_siguientes = lista_indices(siguientes_tokenizadas, palabras_a_indices)

print(siguientes_tokenizadas[0])
print(indices_siguientes[0])

['grupo', 'es', 'tatal', 'elec', 'tri', 'ci', 'te', 'de', 'fran', 'ce', 'e', 'd', 'f', 'anuncio', 'hoy', 'jue', 'ves', 'com', 'pra', '5', '1', 'porciento', 'empresa', 'mexi', 'cana', 'elec', 'tri', 'ci', 'dadagui', 'lade', 'al', 'tami', 'ra', 'e', 'a', 'a', 'cre', 'ada', 'japones', 'mi', 't', 'subis', 'hi', 'cor', 'poration', 'poneren', 'mar', 'cha', 'central', 'gas', '4', '9', '5', 'me', 'ga', 'vatios', '<EOS>']
[270, 17, 1437, 919, 175, 44, 13, 0, 191, 64, 14, 25, 35, 519, 34, 159, 241, 78, 970, 117, 36, 147, 293, 742, 336, 919, 175, 44, 2590, 2590, 16, 801, 48, 14, 1, 1, 151, 737, 2338, 52, 39, 2339, 75, 249, 2340, 1475, 102, 145, 1216, 917, 61, 123, 117, 97, 129, 1650, 2592]


## Paso 2 
Construimos la red neuronal recurrente 

In [19]:
import torch    
import torch.nn as nn

In [60]:
class RecurrentNetwork(nn.Module):
    def __init__(self, dim_in, dim_out, dim=100, dim_h=200):
        super().__init__()
        #Capa de embedding
        self.emb = nn.Embedding(dim_in,dim)
        #Capa de RNN (se toma bidireccional)
        self.recurrence = nn.RNN(dim, dim_h, bidirectional=True)
        #Salida
        self.ffw = nn.Sequential(nn.Linear(2*dim_h,dim_out), nn.Softmax(dim=2))
        
    def forward(self, x):
        #Se pasa a formato torch
        x = torch.tensor(x)
        #Embedding
        x = self.emb(x)
        #Ajustes de tamaño
        x = x.unsqueeze(1)
        #Estados de recurrencia
        h, c = self.recurrence(x)
        #Activación
        h = h.tanh()
        #Salida
        y_pred = self.ffw(h)
        #Se acomoda la salida para que la tome el loss
        y_pred = y_pred.transpose(1, 2)
        
        return y_pred

In [64]:
rnn = RecurrentNetwork(len(indices_a_palabras.keys()), len(indices_a_palabras.keys()))
x = indices_sents
y = indices_siguientes

In [67]:
#Numero de iteraciones
epochs = 1
#La función de riesgo es la entropía cruzada
criterion = torch.nn.CrossEntropyLoss()
#Los parametros que se van a actualizar
optimizer = torch.optim.Adagrad(rnn.parameters(), lr=0.1)

#Se entrena el modelo
for epoch in range(epochs):
    for x_i, y_i in zip(x, y):
        #FORWARD
        y_pred = rnn(x_i)

        #BACKWARD
        #Resize de las variables esperadas (se agrega dimension de length_seq)
        y_i = (torch.tensor(y_i)).unsqueeze(1)
        #Se calcula el eror
        loss = criterion(y_pred, y_i)

        #zero grad
        optimizer.zero_grad()
        #Backprop
        loss.backward()
        #Se actualizan los parametros
        optimizer.step()

    print(f"Epoch={epoch}. Training loss={loss}")


Epoch=0. Training loss=7.787142276763916


In [127]:
output, hn = rnn([270, 17])
#Vocanulario de indice a texto

probas = []

for i in range(len(indices_a_palabras.keys())):
    indices = list(indices_a_palabras.keys())
    palabra = indices_a_palabras[indices[i]]
    valor = output[i].item()
    #print(f"{palabra} = {valor}")
    probas.append((palabra, valor))

def get_proba(tupla):
    return tupla[1]

probas.sort(key=get_proba, reverse=True)
print(probas)



[('<UNK>', 0.9800570011138916), ('<EOS>', 0.019942982122302055), ('millones', 4.72071383719741e-13), ('f', 4.895915487260205e-14), ('graciasa', 2.437403713107314e-14), ('centrales', 1.384686283343355e-17), ('te', 7.956192777871255e-19), ('me', 5.333001430191294e-20), ('elec', 1.2368792117620642e-20), ('de', 8.482959324962209e-21), ('llo', 3.2880603024599744e-21), ('ga', 3.840509616075189e-22), ('t', 3.640896252618064e-22), ('d', 3.219471863355109e-22), ('ciudades', 1.4368026013414733e-22), ('tica', 1.2357836770778619e-22), ('poneren', 1.1863534003212236e-22), ('japonesa', 7.085472288959503e-23), ('años', 4.587552121186221e-23), ('cong', 1.5179357779293146e-23), ('pronun', 1.5173046892051378e-23), ('cha', 1.4475738028046696e-23), ('9', 1.0670320352914848e-23), ('vatios', 7.705887567316491e-24), ('e', 5.1006306460438595e-24), ('ccion', 3.4729976061285025e-24), ('casos', 2.587162227043898e-24), ('iber', 2.2045337252211773e-24), ('tencia', 8.837468698380319e-25), ('po', 8.282421235086157e-